In [5]:
import os
os.listdir("G:\\emcl-data\\.minecraft\\versions\\")

['1.0']

In [11]:
print(input().split(":"))

['i', 'ii', 'iii', 'iiii']


In [6]:
import shutil
print(shutil.get_unpack_formats())

[('bztar', ['.tar.bz2', '.tbz2'], "bzip2'ed tar-file"), ('gztar', ['.tar.gz', '.tgz'], "gzip'ed tar-file"), ('tar', ['.tar'], 'uncompressed tar file'), ('xztar', ['.tar.xz', '.txz'], "xz'ed tar-file"), ('zip', ['.zip'], 'ZIP file')]


In [4]:
for key in [["a","b"],["c","d"]]:
    print(key)

['a', 'b']
['c', 'd']
